> May 06, 2024 | {sub-ref}`wordcount-minutes` min read

::::{figure} https://quantumspain-project.es/wp-content/uploads/2022/11/Logo_QS_EspanaDigital.png
:width: 1000px
:align: left
::::



Authors:
* Irais Bautista ([CESGA](http://www.cesga.es)), Sergio Martínez ([BIFi-UNIZAR](https://bifi.es/es/)), Jaime Scharfhausen ([UAM](https://www.uam.es/uam/inicio)) y Alejandro Jaramillo ([CSUC](https://www.csuc.cat/es))

<img src="https://quantumspain-project.es/wp-content/uploads/2022/11/CESGA.png" width="100px">
<img src="http://bifi.es/wp-content/uploads/2016/11/logo_vectorial-web.png" width="200px">
<img src="https://www.iib.uam.es/iiblf6theme-theme/images/custom/logo-uam.png" width = "200px">
<img src="https://www.csuc.cat/sites/default/files/2021-02/CSUC_logo_corporatiu_0.png" width = "200px">

<a id = 'sec_Ansatz_k-UpCCGSD'></a>
# Ansatz k-UpCCGSD

In [1]:
import qiskit
# from qiskit_ibm_provider import IBMProvider
# from qiskit import IBMQ
# import qiskit.tools.jupyter
from qiskit.tools.parallel import parallel_map
import json
from styles.style import qspain

from qiskit.algorithms.minimum_eigensolvers import VQE
from qiskit.algorithms.optimizers import COBYLA, SPSA, POWELL
from qiskit.primitives import Estimator

from qiskit_nature.second_q.circuit.library import UCC, PUCCD, HartreeFock
from qiskit_nature.second_q.mappers import QubitMapper, JordanWignerMapper
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

import pennylane as qml
from pennylane import numpy as np


%matplotlib inline
#IBMQ.load_account()

#%qiskit_backend_overview

/tmp/ipykernel_7025/537049398.py:9: DeprecationWarning: ``qiskit.algorithms`` has been migrated to an independent package: https://github.com/qiskit-community/qiskit-algorithms. The ``qiskit.algorithms`` import path is deprecated as of qiskit-terra 0.25.0 and will be removed no earlier than 3 months after the release date. Please run ``pip install qiskit_algorithms`` and use ``import qiskit_algorithms`` instead.
  from qiskit.algorithms.minimum_eigensolvers import VQE


:::{contents}
:local:
:depth: 1
:::


(sec_Teoría_de_Coupled_Cluster)= 
## Teoría de Coupled Cluster

Para obtener el estado fundamental de una molécula, dado que son sistemas relativamente complejos en los que los diferentes electrones pueden ocupar muchas posiciones en los diferentes orbitales, es prácticamente imposible obtener un resultado exacto. En su lugar, se utilizan métodos aproximados.


El más famoso de estos métodos, así como el punto de partida para empezar a aplicar el resto, es el método de Hartree-Fock. Con él se pueden aproximar las funciones de onda de los electrones del sistema atendiendo a la configuración de los orbitales atómicos o moleculares. 


Sin embargo, este método está bastante limitado por varios motivos: no considera la correlación electrónica, hace uso de la aproximación de Born-Oppenheimer para los núcleos, no tiene en cuenta posibles efectos relativistas... Es por todo esto que, tras obtener una primera aproximación del estado fundamental, se utilizan otros métodos para terminar de "pulir" este resultado como *Configuration Interaction* (CI) o *Møller–Plesset perturbation theory* (MP) {cite}`bib_andres_bort_theoretical_2007`.


Es dentro de estos métodos post-Hartree-Fock donde encontramos los métodos de *Coupled Cluster* {cite}`bib_cizek_coupled_1980`, {cite}`bib_mcclean_theory_2016`, {cite}`bib_anand_quantum_2022`. La base del CC es, en resumidas cuentas, considerar de forma ponderada cómo contribuyen los estados con electrones a la energía del estado fundamental molecular. Para ello se añaden términos a la función de onda considerando las excitaciones de un electron de los orbitales libres a los orbitales ocupados (*Single excitation*), después de dos electrones con todas las combinaciones posibles (*Double*) y así sucesivamente.


Los métodos CC se usan para crear sistemas correlacionados utilizando un **operador de cluster** $\hat{T}$, de forma que el estado fabricado se genera a partir de otro de referencia haciendo 

$$
\ket{\psi} = e^{\hat{T}} \ket{\phi_0}
$$ 


siendo $\ket{\phi_0}$ el estado de referencia en cuestión, típicamente el de Hartree-Fock.


Dentro de estos métodos encontramos los CCSD (***Single and Double excitations*** *Coupled Cluster*), es decir, métodos que se limitan a considerar excitaciones simples o dobles de los orbitales. Cuando trabajamos con CCSD, el operador de cluster toma la forma 

$$ 
\hat{T} =  \hat{T}_1 + \hat{T}_2 
$$ 


siendo 

$$ 
\hat{T}_1 = \sum_{i,a} t_i^a \hat{a}^\dagger_a \hat{a}_i 
$$


y 

$$  
\hat{T}_2 = \frac{1}{4} \sum_{i,j ; a,b} t^{ab}_{ij} \hat{a}^\dagger_{a} \hat{a}^\dagger_{b} \hat{a}_j \hat{a}_i 
$$


Los índices $i,j$ denotan orbitales ocupados, mientras que los índices $a,b$ denotan orbitales libres. Los parámetros a optimizar por tanto por el algoritmo VQE son precisamente estos $t^a_i, t^{ab}_{ij}$.


Los operadores CC presentan un problema a la hora de implementarlos como parte de un algoritmo VQE, ya que la forma de proyectar el Hamiltoniano del sistema sobre los estados fabricados para calcular la energía no es compatible con el método variacional, que es la columna vertebral del VQE. Para solventar este problema tenemos un caso particular de los métodos CC: los métodos UCC (***Unitary*** *Coupled Cluster*). 


La solución al problema variacional es muy simple: la forma de fabricar el *ansatz* pasa a ser 

$$ 
\ket{\psi} = e^{\hat{T} - \hat{T}^\dagger} \ket{\phi_0} 
$$


Al igual que con los CC, podemos definir métodos UCCSD, con excitaciones simples y dobles, tomando de nuevo $ \hat{T} =  \hat{T}_1 + \hat{T}_2 $. Un ordenador cuántico puede implementar eficientemente estos métodos UCCSD utilizando una **trotterización** del operador de cluster exponenciado.


Después de esto, podemos buscar otro caso de UCC, el llamado UCCG (*Unitary Coupled Cluster **with Generalized wave functions***), el cual no distingue entre orbitales ocupados y libres. Es por ello que, en el caso UCCGSD, el operador de cluster vuelve a estar definido como $ \hat{T} = \hat{T}_1 + \hat{T}_2 $, pero esta vez 

$$ 
\hat{T}_1 = \frac{1}{2} \sum_{p,q} t^q_p \hat{a}^\dagger_q \hat{a}_p 
$$ 


y 

$$ 
\hat{T}_2 = \frac{1}{4} \sum_{p,q,r,s} t^{rs}_{pq} \hat{a}^\dagger_r \hat{a}^\dagger_s \hat{a}_q \hat{a}_p 
$$ 


denotando los cuatro índices $p,q,r,s$ cualquier tipo de orbital sin ocupar su ocupación.


Estos últimos *ansätze* se pueden implementar en qiskit utilizando ```qiskit_nature```.

De aquí podemos sacar una clase un poco más restringida, la clase UpCCGSD (*Unitary **Pair** Coupled Cluster with Generalized Singles and Doubles product wave functions*), que limita las amplitudes de las excitaciones dobles considerando únicamente las que mueven **un par de electrones** de un orbital espacial a otro. Esto provoca un cambio en el operador $\hat{T}_2$, que se convierte en 

$$ 
\hat{T}_2 = \sum_{p,q} t^{q_\alpha q_\beta}_{p_\alpha p_\beta} \hat{a}^\dagger_{q_\alpha} \hat{a}^\dagger_{q_\beta} \hat{a}_{p_\beta} \hat{a}_{p_\alpha}
$$


Y de aquí por fin llegamos a la clase que queremos estudiar, k-UpCCGSD {cite}`bib_lee_generalized_2019`, {cite}`bib_greene_diniz_generalized_2021`, {cite}`bib_lim_quantum_2022`, con $k \in \mathbb{Z}$. Con este entero $k$ definimos nuestra función de prueba $\ket{\psi}$ como 

$$ 
\ket{\psi} = \left( \prod_{\mu = 1}^k e^{\hat{T}^{(\mu)} - \hat{T}^{(\mu) \dagger}} \right) \ket{\phi_0} 
$$ 


haciendo que cada $ \hat{T}^{(\mu)} $ tenga su propio conjunto de parámetros $t^q_p, t^{q_\alpha q_\beta}_{p_\alpha p_\beta}$. 

(sec_Utilizar_k-UpCCGSD)= 
## Utilizar k-UpCCGSD

Las mejores alternativas para implementar el ansatz k-UpCCGSD son ```qiskit```, librería con la que ya estamos, familiarizados, y ```pennylane``` otra librería enfocada a la Computación Cuántica y, más concretamente, a Quantum Machine Learning.


Como ejemplo prototípico para problemas moleculares, vamos a resolver la molécula de hidrógeno $H_2$.

:::::{tab-set}
::::{tab-item} Pennylane
```python

# Construimos la molécula de Hidrógeno H2
symbols = ["H", "H"]
coordinates = np.array([0.0, 0.0, 0.0, 0.0, 0.0, 1.400])
H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates)

# Obtenemos el estado de referencia de Hartree-Fock
electrons = 2
ref_state = qml.qchem.hf_state(electrons, qubits)
```
::::
::::{tab-item} Qiskit
```python

# Construimos la molécula de Hidrógeno H2
driver = PySCFDriver(
    atom="H 0 0 0; H 0 0 0.741",
    basis="sto3g",
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
    # El método de aproximación por defecto es Hartree-Fock: method=MethodType.RHF
)

es_problem = driver.run()

# Utilizamos el mapping de Jordan-Wigner
mapper = JordanWignerMapper()

# Obtenemos el estado de referencia de Hartree-Fock
ref_state = HartreeFock(es_problem.num_spatial_orbitals, es_problem.num_particles, mapper)
```
::::
:::::


In [2]:
'''Pennylane_code_cell'''

# Construimos la molécula de Hidrógeno H2
symbols = ["H", "H"]
coordinates = np.array([0.0, 0.0, 0.0, 0.0, 0.0, 1.400])
H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates)

# Obtenemos el estado de referencia de Hartree-Fock
electrons = 2
ref_state = qml.qchem.hf_state(electrons, qubits)

Un detalle importante a tener en cuenta a la hora de trabajar con estas librerías es que, mientras que en ```qiskit``` podemos decidir con qué unidades trabajamos, en ```pennylane``` trabajamos con unidades atómicas.En estas unidades, la distancia de enlace entre los átomos de hidrógeno de la molécula es $ 0.741 \cdot 10^{-10} \text{m} =1.40 a_0$

:::::{tab-set}
::::{tab-item} Pennylane
```python

# En pennylane tenemos que definir el circuito sobre el que trabajará el ansatz
dev = qml.device('default.qubit', wires=qubits)

# Definimos el ansatz k-UpCCGSD
# weights son los parámetros t^q_p, t^{q_\alpha q_\beta}_{p_\alpha p_\beta}
# wires son los qubits (o en general los sistemas) sobre los que se define el ansatz.
# delta_sz especifica las reglas de selección para la proyección del espín en dirección z 
#     en los orbitales de las excitaciones simples.

k = 3

@qml.qnode(dev)
def ansatz(weights):
    qml.kUpCCGSD(weights, wires=[0, 1, 2, 3],
                    k=k, delta_sz=0, init_state=ref_state)
    return qml.expval(H)

shape = qml.kUpCCGSD.shape(k=k,
                    n_wires=qubits, delta_sz=0)
```
::::
::::{tab-item} Qiskit
```python

# Preparamos el ansatz k-UpCCGSD

k = 3

# Para preparar este ansatz partimos de una función que ejecuta un ansatz UpCCD
# Podemos añadirle las k repeticiones con la opción reps
# Podemos añadirle las excitaciones S con la opción include_singles
# Podemos añadirle las excitaciones generalizadas con la opción generalized

ansatz = PUCCD(
    num_spatial_orbitals=es_problem.num_spatial_orbitals, 
    num_particles=es_problem.num_particles, 
    qubit_mapper=mapper, 
    include_singles=(True,True), 
    generalized=True, 
    reps=k
)
```
::::
:::::


In [3]:
'''Pennylane_code_cell'''

# En pennylane tenemos que definir el circuito sobre el que trabajará el ansatz
dev = qml.device('default.qubit', wires=qubits)

# Definimos el ansatz k-UpCCGSD
# weights son los parámetros t^q_p, t^{q_\alpha q_\beta}_{p_\alpha p_\beta}
# wires son los qubits (o en general los sistemas) sobre los que se define el ansatz.
# delta_sz especifica las reglas de selección para la proyección del espín en dirección z 
#     en los orbitales de las excitaciones simples.

k = 3

@qml.qnode(dev)
def ansatz(weights):
    qml.kUpCCGSD(weights, wires=[0, 1, 2, 3],
                    k=k, delta_sz=0, init_state=ref_state)
    return qml.expval(H)

shape = qml.kUpCCGSD.shape(k=k,
                    n_wires=qubits, delta_sz=0)

Lo siguiente que tenemos que hacer es escoger el optimizador clásico.

:::::{tab-set}
::::{tab-item} Pennylane
```python

# Escogemos un optimizador
opt = qml.GradientDescentOptimizer(stepsize=0.4)
# opt = qml.GradientDescentOptimizer(stepsize=0.04)
# opt = qml.QNGOptimizer(stepsize=0.01)
# opt = qml.RMSPropOptimizer(stepsize=0.01)
# opt = qml.AdamOptimizer(stepsize=0.01)
# opt = qml.AdagradOptimizer(stepsize=0.01)
# opt = qml.SPSAOptimizer(maxiter=250) 
```
::::
::::{tab-item} Qiskit
```python

# Escogemos un optimizador
opt = COBYLA(maxiter=1000)
# opt = POWELL()
```
::::
:::::


In [4]:
'''Pennylane_code_cell'''

# Escogemos un optimizador
opt = qml.GradientDescentOptimizer(stepsize=0.4)
# opt = qml.GradientDescentOptimizer(stepsize=0.04)
# opt = qml.QNGOptimizer(stepsize=0.01)
# opt = qml.RMSPropOptimizer(stepsize=0.01)
# opt = qml.AdamOptimizer(stepsize=0.01)
# opt = qml.AdagradOptimizer(stepsize=0.01)
# opt = qml.SPSAOptimizer(maxiter=250) 

Y ya podemos resolver el problema VQE

:::::{tab-set}
::::{tab-item} Pennylane
```python

# Inicializamos los primeros parámetros
weights = np.random.random(size=shape)
energy = [ansatz(weights)]

# E iniciamos el proceso iterativo
angle = [weights]
max_iterations = 250
conv_tol = 1e-08 # Escogemos una tolerancia en la que queremos que se detenga el algoritmo
for n in range(max_iterations):
    weights, prev_energy = opt.step_and_cost(ansatz, weights)
    energy.append(ansatz(weights))
    angle.append(weights)
    conv = np.abs(energy[-1] - prev_energy)
    # if n % 4 == 0:
        # print(f"Iteración: {n},  Energía = {energy[-1]:.8f} Ha")
    if conv <= conv_tol:
        break

print("\n" f"Energía del estado fundamental (obtenida) = {energy[-1]:.8f} Ha")
print("\n" f"Valores óptimos del circuito = {angle[-1]}")
```
::::
::::{tab-item} Qiskit
```python

# Ejecutamos el algoritmo
vqe_solver = VQE(Estimator(), ansatz, opt)

calc = GroundStateEigensolver(mapper, vqe_solver)

res = calc.solve(es_problem)
best = res.total_energies
print('E_G =',best)
```
::::
:::::


In [5]:
'''Pennylane_code_cell'''

# Inicializamos los primeros parámetros
weights = np.random.random(size=shape)
energy = [ansatz(weights)]

# E iniciamos el proceso iterativo
angle = [weights]
max_iterations = 250
conv_tol = 1e-08 # Escogemos una tolerancia en la que queremos que se detenga el algoritmo
for n in range(max_iterations):
    weights, prev_energy = opt.step_and_cost(ansatz, weights)
    energy.append(ansatz(weights))
    angle.append(weights)
    conv = np.abs(energy[-1] - prev_energy)
    # if n % 4 == 0:
        # print(f"Iteración: {n},  Energía = {energy[-1]:.8f} Ha")
    if conv <= conv_tol:
        break

print("\n" f"Energía del estado fundamental (obtenida) = {energy[-1]:.8f} Ha")
print("\n" f"Valores óptimos del circuito = {angle[-1]}")


Energía del estado fundamental (obtenida) = -1.13727592 Ha

Valores óptimos del circuito = [[0.47277375 0.763124   0.65265082 0.80044558 0.63689578 0.2509901 ]
 [0.41734635 0.41822243 0.85411198 0.21763306 0.31178434 0.49424296]
 [0.88739316 0.52998541 0.2680871  0.68472122 0.53887998 0.5154465 ]]


Mostramos aquí algunas tablas del valor al que converge la energía utilizando diferentes optimizadores y valores de $k$.


|k|E (Ha) GradDesc|E (Ha) SPSA|E (Ha) ADAM|E (Ha) RMSProp|E (Ha) Adargrad| E (Ha) QNG|
|---|---|---|---|---|---|---|
|3|-1.13727592|-1.13575585|-1.13727562|-1.13628251|-1.1372758 |-1.13727591|
|4|-1.13727592|-1.1355345 |-1.13727573|-1.13727594|-1.13727586|-1.13727586|
|5|-1.13727585|-1.13606927|-1.13726938|-1.13465174|-1.1372759 |-1.13727585|
|6|-1.13726475|-1.13358301|-1.13727593|-1.13727594|-1.1372757 |-1.13727587|
|7|-1.13727592|-1.13560152|-1.13727591|-1.13339437|-1.13727577|-1.1372759 |

---
## Bibliografía 
```{bibliography} 
:style: plain
:filter: docname in docnames
```

---
<center>
<a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/">
<img aling="left" alt="Licencia Creative Commons" style="border-width:0" src="https://i.creativecommons.org/l/by-sa/4.0/88x31.png" /></a>
</center>

License: <a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/">Licencia Creative Commons Atribución-CompartirIgual 4.0 Internacional</a>.

This work has been financially supported by the Ministry for Digital Transformation and of Civil Service of the Spanish Government through the QUANTUM ENIA project call - Quantum Spain project, and by the European Union through the Recovery, Transformation and Resilience Plan - NextGenerationEU within the framework of the Digital Spain 2026 Agenda.

<img align="left" src=" https://quantumspain-project.es/wp-content/uploads/2024/02/Banner-QS_GOB_v2.png" width="1000px" />